In [1]:
#describe each location with companies in side
import pandas as pd
import numpy as np
import os
import pygeohash as pgh
from math import *
from sklearn.metrics.pairwise import euclidean_distances
import random
pjoin = os.path.join

In [2]:
#data load
datapath = '/Users/yefeichen/Database/location_recommender_system/'
cfile = ['dnb_pa.csv','dnb_sf.csv','dnb_sj.csv']
lfile = 'location_scorecard_190912.csv'
clfile = ['PA.csv','SF.csv','SJ.csv']

pdc1 = pd.read_csv(pjoin(datapath,cfile[0]))
# pdc2 = pd.read_csv(pjoin(datapath,cfile[1]))
# pdc3 = pd.read_csv(pjoin(datapath,cfile[2]))

# pdc = pd.concat([pdc1,pdc2,pdc3],axis=0)
pdl = pd.read_csv(pjoin(datapath,lfile))

pdcl1 = pd.read_csv(pjoin(datapath,clfile[0]))
# pdcl2 = pd.read_csv(pjoin(datapath,clfile[1]))
# pdcl3 = pd.read_csv(pjoin(datapath,clfile[2]))



In [3]:
#create training data and testing data
def splitdat(dat,key_column=['duns_number'],right_colunm='atlas_location_uuid_tr',rate_tr=0.8):
    """
    split the <company,location> pair into training/testing dat
    """
    tr = dat.sample(frac=rate_tr)
    tt = pd.merge(dat,tr,on=key_column,how='left',suffixes=['','_tr'])
    tt = tt[tt[right_colunm].isnull()]
    tt = tt[list(tr.columns)]
    print('Train dat:', len(tr), 'Test dat:', len(tt))
    return tr,tt

In [4]:
tr1,tt1 = splitdat(pdcl1)
# tr2,tt2 = splitdat(pdcl2)
# tr3,tt3 = splitdat(pdcl3)
tr1['duns_number']=tr1['duns_number'].astype(int)

# tr = pd.concat([tr1,tr2,tr3],axis=0)
# tr['duns_number']=tr['duns_number'].astype(int)

Train dat: 4975 Test dat: 1244


In [6]:
#data process
def onehotdat(dat,key_column:list):
    dat[key_column] = dat[key_column].astype(str)
    dum_dat = pd.get_dummies(dat[key_column],dummy_na=True)
    return dum_dat

def split2num(emp_range:str):
    max_emp_val = emp_range.replace(' ','').split('-')
    if len(max_emp_val)<2:
        return 10
    else:
        return float(max_emp_val[1])
    
def max_col(dat,col,minval=1):
    dat[col] = dat[col].apply(lambda r:max(r,minval))

def comp_dat_process(dat):
    """
    pd -> company key,cont_feature,spec_feature,dum_feature
    """
    one_hot_col_name = ['major_industry_category','location_type','primary_sic_2_digit']
    spec_col_name = 'emp_here_range'
    cont_col_name = ['emp_here','emp_total','sales_volume_us','square_footage']

    print('doing one-hot...')
    dum_dat = onehotdat(dat,one_hot_col_name)
    
    print('extract continuous...')
    cont_dat = dat[cont_col_name].fillna(value=0).astype(float)
    
    print('specific feature')
    spec_dat = dat[spec_col_name].fillna(value='1-10').astype(str)
    spec_dat = spec_dat.apply(lambda row: split2num(row))
    
    max_col(cont_dat,'emp_here',1)
    
    res_dat = dat[['duns_number']].join([cont_dat,spec_dat,dum_dat],how='left')
    assert(len(res_dat)==len(dum_dat))
    assert(len(res_dat)==len(cont_dat))
    assert(len(res_dat)==len(spec_dat))
    return res_dat
    
proc_pdc1 = comp_dat_process(pdc1)
print(len(proc_pdc1))
proc_pdc1.head()

doing one-hot...
extract continuous...
specific feature
7538


,duns_number,emp_here,emp_total,sales_volume_us,square_footage,emp_here_range,major_industry_category_AGRICULTURE/FORESTRY/FISHING,major_industry_category_CONSTRUCTION,major_industry_category_FIN/INSURANCE/REAL ESTATE,major_industry_category_MANUFACTURING,...,primary_sic_2_digit_86,primary_sic_2_digit_87,primary_sic_2_digit_89,primary_sic_2_digit_91,primary_sic_2_digit_92,primary_sic_2_digit_94,primary_sic_2_digit_95,primary_sic_2_digit_96,primary_sic_2_digit_97,primary_sic_2_digit_nan
0,105571242,1.0,1.0,28287.0,1678.0,4.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,555658061,1.0,1.0,83843.0,1703.0,4.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,36779789,10.0,10.0,122789.0,5111.0,14.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,62393456,1.0,1.0,75000.0,1438.0,4.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,60656896,3.0,3.0,135306.0,6124.0,4.0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
def comp_transpd2np(featdat,trdat,ttdat):
    tr_feat = pd.merge(trdat,featdat,on='duns_number',how='inner')
    col_list = list(tr_feat.columns)
#     print(col_list)
    trainX = tr_feat.loc[:,col_list[3:]].to_numpy()
    trainY = tr_feat[['atlas_location_uuid','longitude_loc','latitude_loc']].to_numpy()
    
    tt_feat = pd.merge(ttdat,featdat,on='duns_number',how='inner')
    col_list = list(tt_feat.columns)
#     print(col_list)
    testX = tt_feat.loc[:,col_list[3:]].to_numpy()
    testY = tt_feat[['atlas_location_uuid','longitude_loc','latitude_loc']].to_numpy()
    return trainX,trainY,testX,testY

In [8]:
trX,trY,ttX,ttY = comp_transpd2np(proc_pdc1,tr1,tt1)

print(ttX.shape)

(1244, 94)


array([['77c493ec-1424-6d74-8db3-ee8fce0092db', -122.158983, 37.4494339],
       ['b89c3096-5ebc-7491-2de8-79f22da62dbf', -122.1287787, 37.4334557],
       ['09ba3f0d-f144-7f40-e986-74ba9d62c747', -122.1599861, 37.445163],
       ...,
       ['c5b270a7-6368-6e77-6039-618cc709f3a6', -122.1271801, 37.4323145],
       ['a0e26081-e098-a151-67d3-9b7194c5e467', -122.1235118, 37.4112621],
       ['c5b270a7-6368-6e77-6039-618cc709f3a6', -122.1271801, 37.4323145]],
      dtype=object)

In [9]:
def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis), l2

In [10]:
#accurate matching
distQR = euclidean_distances(ttX,trX)
print(distQR.shape)

ind = np.argmin(distQR,axis=1)
predY = trY[ind,:]

r,c = predY.shape
cnt = 0
for i in range(0,r):
    if predY[i,0]==ttY[i,0]:
        cnt+=1
        
print(cnt)
print(r)

(1244, 4975)
49
1244


In [11]:
#distance matching
def geo_distance(lng1,lat1,lng2,lat2):
    lng1, lat1, lng2, lat2 = map(radians, [lng1, lat1, lng2, lat2])
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2 
    dis=2*asin(sqrt(a))*6371*1000
    return dis

r,c = predY.shape
cnt = 0
dist = []
for i in range(0,r):
    distance = geo_distance(predY[i,1],predY[i,2],ttY[i,1],ttY[i,2])
    dist.append(distance)
    



In [12]:
K = np.array(dist)
K2 = np.percentile(K,[25,50,75])
K2

array([ 963.16516775, 2610.41825039, 3648.42077032])

In [314]:
def normalize_dat(trX,ttX,cols=5,axis=0):
    D = trX[:,:cols]
    center = D.mean(axis=axis)
    center = np.expand_dims(center,axis)
    scale = D.std(axis=axis)
    scale = np.expand_dims(scale,axis)
    
    trX[:,:cols] = (D-center)/scale
    ttX[:,:cols] = (ttX[:,:cols]-center)/scale
    

normalize_dat(trX,ttX,cols=5)

In [65]:
comp_feat_name = ['duns_number','latitude','longitude','emp_here','emp_here_range','emp_total','sales_volume_us','major_industry_category','square_footage','primary_sic_2_digit','naics_2_digit']
loc_feat_name = ['atlas_location_uuid','latitude','longitude','city','is_wework','building_class','num_emp_weworkcore','num_poi_weworkcore','pct_wwcore_employee','pct_wwcore_business','num_retail_stores','num_doctor_offices','num_eating_places','num_drinking_places','num_hotels','num_fitness_gyms','population_density','pct_female_population','median_age','income_per_capita','pct_masters_degree','walk_score','bike_score']
print(len(comp_feat_name),len(loc_feat_name))

11 23
